In [1]:
import glob
import pandas as pd
import json
import requests

## Importing and Loading the datasets

In [2]:
df1 = pd.read_csv('C:/Users/Srinivasa Rao/OneDrive/Documents/GUVI_Projects/Singapore_House_Resale/ResaleFlatPricesBasedonApprovalDate19901999.csv')
df2 = pd.read_csv("C:/Users/Srinivasa Rao/OneDrive/Documents/GUVI_Projects/Singapore_House_Resale/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
df3 = pd.read_csv("C:/Users/Srinivasa Rao/OneDrive/Documents/GUVI_Projects/Singapore_House_Resale/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")
df4 = pd.read_csv("C:/Users/Srinivasa Rao/OneDrive/Documents/GUVI_Projects/Singapore_House_Resale/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
df5 = pd.read_csv("C:/Users/Srinivasa Rao/OneDrive/Documents/GUVI_Projects/Singapore_House_Resale/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
df = pd.concat([df1 , df2 , df3, df4, df5],axis=0)
df.head()
df


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
181561,2024-05,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,154.0,Maisonette,1988,948000.0,62 years 09 months
181562,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1988,1000000.0,62 years 08 months
181563,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,1000000.0,62 years 08 months
181564,2024-02,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,04 TO 06,164.0,Multi Generation,1987,998000.0,62 years 11 months


In [3]:
df.shape

(927769, 11)

###  Handling Null Values

In [4]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [5]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

### MRT (Mass Rapid Transit System) Railway Transportation
- Getting MRT coordinates for each city
- so that we can calculate the distance later using MRT coordinates

In [6]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [7]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [8]:
mrt_lat = []
mrt_long = []

for query_address in list_of_mrt:
    try:
        query_string = f'https://photon.komoot.io/api/?q={query_address}'
        resp = requests.get(query_string)
        resp.raise_for_status()  # Raise an exception for bad responses (e.g., 404, 500)

        data_mrt = resp.json()

        if data_mrt['features']:
            first_result = data_mrt['features'][0]['geometry']['coordinates']
            mrt_lat.append(first_result[1])  # Latitude
            mrt_long.append(first_result[0])  # Longitude

            print(f"{query_address}, Lat: {first_result[1]} Long: {first_result[0]}")
        else:
            mrt_lat.append('NoResults')
            mrt_long.append('NoResults')
            print(f"No Results for {query_address}")

    except Exception as e:
        print(f"Error fetching data for {query_address}: {e}")
        mrt_lat.append('Error')
        mrt_long.append('Error')

Jurong East MRT Station, Lat: 1.34425925 Long: 103.72242385216289
Bukit Batok MRT Station, Lat: 1.3520257 Long: 103.7657271389443
Bukit Gombak MRT Station, Lat: 1.3591396 Long: 103.7517111
Choa Chu Kang MRT Station, Lat: 1.3978193 Long: 103.7473471
Yew Tee MRT Station, Lat: 1.3978193 Long: 103.7473471
Kranji MRT Station, Lat: 3.1240029500000004 Long: 101.72736629198937
Marsiling MRT Station, Lat: 3.1240029500000004 Long: 101.72736629198937
Woodlands MRT Station, Lat: 1.4403443 Long: 103.8008888
Admiralty MRT Station, Lat: 1.4403443 Long: 103.8008888
Sembawang MRT Station, Lat: 3.1240029500000004 Long: 101.72736629198937
Canberra MRT Station, Lat: 3.1240029500000004 Long: 101.72736629198937
Yishun MRT Station, Lat: 3.1240029500000004 Long: 101.72736629198937
Khatib MRT Station, Lat: 1.4173799 Long: 103.8329514
Yio Chu Kang MRT Station, Lat: 1.3826263 Long: 103.8442871
Ang Mo Kio MRT Station, Lat: 56.7292945 Long: -2.4521729
Bishan MRT Station, Lat: 3.1240029500000004 Long: 101.727366291

In [9]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

In [10]:
mrt_location.to_csv('mrt.csv')

- Getting coordinates of each HDB(Housing and Developing Board) Resale flat in order to conduct the distance from MRT stations
- So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System) and also from CBD (Central Business District).

In [11]:
df['address'] = df['block'] + " " + df['street_name']
df

C:\Users\Srinivasa Rao\AppData\Local\Temp\ipykernel_15180\2900962711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['address'] = df['block'] + " " + df['street_name']


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,174 ANG MO KIO AVE 4
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65,541 ANG MO KIO AVE 10
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64,163 ANG MO KIO AVE 4
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63,446 ANG MO KIO AVE 10
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
181561,2024-05,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,154.0,Maisonette,1988,948000.0,62 years 09 months,834 YISHUN ST 81
181562,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1988,1000000.0,62 years 08 months,826 YISHUN ST 81
181563,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,1000000.0,62 years 08 months,826 YISHUN ST 81
181564,2024-02,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,04 TO 06,164.0,Multi Generation,1987,998000.0,62 years 11 months,666 YISHUN AVE 4


In [12]:
address_list = df['address'].unique()

In [17]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []

for query_address in address_list:
    try:
        query_string = f'https://photon.komoot.io/api/?q={query_address}'
        resp = requests.get(query_string)
        resp.raise_for_status()  # Raise an exception for bad responses (e.g., 404, 500)

        data_geo_location = resp.json()

        if data_geo_location['features']:
            result = data_geo_location['features'][0]
            geometry = result['geometry']['coordinates']
            properties = result['properties']

            latitude.append(geometry[1])  # Latitude
            longitude.append(geometry[0])  # Longitude
            blk_no.append(properties.get('housenumber', ''))
            road_name.append(properties.get('street', ''))
            postal_code.append(properties.get('postcode', ''))
            address.append(query_address)

            print(f"{query_address}, Lat: {geometry[1]} Long: {geometry[0]}")
        else:
            latitude.append('NoResults')
            longitude.append('NoResults')
            blk_no.append('')
            road_name.append('')
            postal_code.append('')
            address.append(query_address)
            print(f"No Results for {query_address}")

    except Exception as e:
        print(f"Error fetching data for {query_address}: {e}")
        latitude.append('Error')
        longitude.append('Error')
        blk_no.append('')
        road_name.append('')
        postal_code.append('')
        address.append(query_address)

204D COMPASSVALE DR, Lat: 1.38761245 Long: 103.89517595634311
105 SERANGOON NTH AVE 1, Lat: 1.3702561 Long: 103.870233
107 SERANGOON NTH AVE 1, Lat: 1.36991005 Long: 103.8702301250627
115 SERANGOON NTH AVE 1, Lat: 1.3682576000000002 Long: 103.87077024317026
4 LOR LEW LIAN, Lat: 1.35024185 Long: 103.87644435575413
219 SERANGOON AVE 4, Lat: 1.35837905 Long: 103.87220995
1 LOR LEW LIAN, Lat: 1.3508851 Long: 103.87534199786184
225 SERANGOON AVE 4, Lat: 1.3570181 Long: 103.8714251
223 SERANGOON AVE 4, Lat: 1.3577742499999998 Long: 103.8707064
209 SERANGOON CTRL, Lat: 1.3545011 Long: 103.87421022621999
323 SERANGOON AVE 3, Lat: 1.3513177 Long: 103.86950343644438
2 LOR LEW LIAN, Lat: 1.35047825 Long: 103.87545689574871
101 SERANGOON NTH AVE 1, Lat: 1.3710233 Long: 103.8703865164769
127 SERANGOON NTH AVE 1, Lat: 1.36565595 Long: 103.87184133003237
547 SERANGOON NTH AVE 3, Lat: 1.3739139 Long: 103.87038257255855
7 LOR LEW LIAN, Lat: 1.3492038499999999 Long: 103.87701555000001
216 SERANGOON AVE 

In [18]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

9590

In [26]:
df_coordinates = df_coordinates[df_coordinates['latitude'] != 'NoResults']
df_coordinates = df_coordinates[df_coordinates['latitude'] != 'Error']


In [27]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [28]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

## Getting the distance with the help of OpenMap API

In [29]:
# Distance to nearest MRT
!pip install geopy

from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [31]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [32]:
df_coordinates

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt
0,1.369208,103.850399,,Ang Mo Kio Avenue 3,560424,174 ANG MO KIO AVE 4,9532.944138,1632.207223
1,1.373984,103.855591,541,Ang Mo Kio Avenue 10,560541,541 ANG MO KIO AVE 10,10071.820645,1579.796911
2,1.373846,103.838589,163,Ang Mo Kio Avenue 4,560163,163 ANG MO KIO AVE 4,10144.423617,1159.637665
3,1.367779,103.855334,446,Ang Mo Kio Avenue 10,560446,446 ANG MO KIO AVE 10,9385.217269,2051.015640
4,1.371655,103.857756,557,Ang Mo Kio Avenue 10,560557,557 ANG MO KIO AVE 10,9829.270327,1928.355483
...,...,...,...,...,...,...,...,...
9585,1.36082,103.93698,601A,Tampines Avenue 9,521601,601A TAMPINES AVE 9,12843.925759,895.347518
9586,1.361395,103.935684,604B,Tampines Avenue 9,522604,604B TAMPINES AVE 9,12780.295820,791.796712
9587,1.361354,103.937285,601C,Tampines Avenue 9,521601,601C TAMPINES AVE 9,12908.768977,949.384213
9588,1.362779,103.939227,,,522618,606D TAMPINES ST 61,13174.723506,1211.995411


In [33]:
df_coordinates.to_csv('df_coordinates.csv',index=False)

In [34]:
df_coordinates = pd.read_csv('df_coordinates.csv')

In [35]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,10.092052,-61.578033,1,BEACH RD,NaN,1 BEACH RD,1.797767e+07,3.230714e+06,2015-01,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,04 TO 06,74.0,Improved,1979,388000.0,63
1,10.092052,-61.578033,1,BEACH RD,NaN,1 BEACH RD,1.797767e+07,3.230714e+06,2016-03,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979,350000.0,62
2,10.092052,-61.578033,1,BEACH RD,NaN,1 BEACH RD,1.797767e+07,3.230714e+06,2017-04,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979,340000.0,61 years 05 months
3,10.092052,-61.578033,1,BEACH RD,NaN,1 BEACH RD,1.797767e+07,3.230714e+06,2017-05,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979,322000.0,61 years 05 months
4,10.092052,-61.578033,1,BEACH RD,NaN,1 BEACH RD,1.797767e+07,3.230714e+06,2017-06,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979,370000.0,61 years 03 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218714,NaN,NaN,NaN,NaN,NaN,9B BOON TIONG RD,NaN,NaN,2023-02,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,25 TO 27,112.0,Improved,2016,1290000.0,91 years 11 months
218715,NaN,NaN,NaN,NaN,NaN,9B BOON TIONG RD,NaN,NaN,2023-05,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,07 TO 09,112.0,Improved,2016,1278000.0,91 years 09 months
218716,NaN,NaN,NaN,NaN,NaN,9B BOON TIONG RD,NaN,NaN,2023-08,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,22 TO 24,112.0,Improved,2016,1325000.0,91 years 05 months
218717,NaN,NaN,NaN,NaN,NaN,9B BOON TIONG RD,NaN,NaN,2023-12,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,19 TO 21,112.0,Improved,2016,1400000.0,91 years 02 months


In [36]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,lease_remain_years
62,1.324671,103.929167,#B1-71/72,"311 New Upper Changi Rd, Bedok Mall",467360,1 CHAI CHEE RD,9814.740577,1870.104491,2015-06,BEDOK,4 ROOM,1,CHAI CHEE RD,10 TO 12,103.0,Model A,1983,445000.0,67,59
63,1.324671,103.929167,#B1-71/72,"311 New Upper Changi Rd, Bedok Mall",467360,1 CHAI CHEE RD,9814.740577,1870.104491,2015-08,BEDOK,5 ROOM,1,CHAI CHEE RD,07 TO 09,133.0,Model A,1983,572500.0,67,59
64,1.324671,103.929167,#B1-71/72,"311 New Upper Changi Rd, Bedok Mall",467360,1 CHAI CHEE RD,9814.740577,1870.104491,2016-02,BEDOK,5 ROOM,1,CHAI CHEE RD,07 TO 09,132.0,Model A,1983,550000.0,66,59
65,1.324671,103.929167,#B1-71/72,"311 New Upper Changi Rd, Bedok Mall",467360,1 CHAI CHEE RD,9814.740577,1870.104491,2016-08,BEDOK,5 ROOM,1,CHAI CHEE RD,04 TO 06,132.0,Model A,1983,540000.0,66,59
66,1.324671,103.929167,#B1-71/72,"311 New Upper Changi Rd, Bedok Mall",467360,1 CHAI CHEE RD,9814.740577,1870.104491,2017-12,BEDOK,5 ROOM,1,CHAI CHEE RD,04 TO 06,132.0,Model A,1983,510000.0,64 years 08 months,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218663,1.339018,103.847442,99C,Lorong 2 Toa Payoh,312099,99C LOR 2 TOA PAYOH,6209.019377,653.763827,2018-06,TOA PAYOH,EXECUTIVE,99C,LOR 2 TOA PAYOH,04 TO 06,148.0,Apartment,1993,888000.0,73 years 10 months,69
218664,1.339018,103.847442,99C,Lorong 2 Toa Payoh,312099,99C LOR 2 TOA PAYOH,6209.019377,653.763827,2021-03,TOA PAYOH,EXECUTIVE,99C,LOR 2 TOA PAYOH,07 TO 09,149.0,Apartment,1993,985000.0,70 years 11 months,69
218665,1.339018,103.847442,99C,Lorong 2 Toa Payoh,312099,99C LOR 2 TOA PAYOH,6209.019377,653.763827,2021-11,TOA PAYOH,EXECUTIVE,99C,LOR 2 TOA PAYOH,10 TO 12,149.0,Apartment,1993,1000000.0,70 years 06 months,69
218666,1.339018,103.847442,99C,Lorong 2 Toa Payoh,312099,99C LOR 2 TOA PAYOH,6209.019377,653.763827,2023-01,TOA PAYOH,EXECUTIVE,99C,LOR 2 TOA PAYOH,07 TO 09,145.0,Apartment,1993,1030000.0,69 years 04 months,69


## Final Combined Data

In [37]:
df_new.to_csv('combined.csv', index = False)

In [38]:
from IPython.display import FileLink

# Create a download link for the CSV file
FileLink('combined.csv')


c:\Users\Srinivasa Rao\OneDrive\Documents\GUVI_Projects\Singapore_House_Resale\combined.csv

In [39]:
from IPython.display import FileLink

# Create a download link for the CSV file
FileLink('df_coordinates.csv')


c:\Users\Srinivasa Rao\OneDrive\Documents\GUVI_Projects\Singapore_House_Resale\df_coordinates.csv